In [3]:
#Install and import the following packages

import circle_fit as cf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_circles
from math import pi,sqrt, fsum, radians


In [4]:
#function reads pts file and return a 2-d numpy array of points
def read_points_file(ptsfile_name):
    points_list = []
    with open(ptsfile_name) as f:
        for line in f:
            if "{" in line or "version" in line or "n_points" in line:
                continue        # Ignore tags
            if line.isspace():
                continue        # Ignore any empty lines.
            if "}" in line:
                break           # Terminate when we hit the enclosing bracket at the end of the IMAGES section
            x, y = line.split()
            points_list.append([x,y])
    return np.asarray(points_list, dtype=np.float32) 
            

In [5]:
#function writes a 2-d numpy array of points to a newly-created file. 
def create_points_file(ptsfile_name, points):
    f = open(ptsfile_name,'w')
    f.write('version: 1' + '\n')
    f.write('n_points: ' + str(len(points)) + '\n')
    f.write('{\n')
    for i in range(len(points)):
        f.write(str(points[i][0])+' '+ str(points[i][1])+'\n')
    f.write('}')
    f.close()
        

In [6]:
#function returns the shortest distance between point x and the line segment ab.
def shortest_distance_from_point_to_segment(x,a, b):
    u = (b-a) / np.linalg.norm(b-a)
    c= np.dot(u,x-a)
    r= np.linalg.norm(x-a)
    v= np.linalg.norm(b-a)
    l=0
    if c < 0:
        d= r
        l= -1
        z = a
    elif c > v:
        d= np.linalg.norm(x-b)
        l= 1
        z = b
    else: 
        d= sqrt(r*r-c*c) # distance
        z = a + c*u # point
    
    return d,l,z

In [7]:
#function to fit circle to points 15-28 of the femoral head using a least squares method
def fit_circle(points):
    head = points[15:29]
    xc, yc, r, s = cf.least_squares_circle(head)
# x of centre, y of centre, r radius of best fit circle, s variance or residual float  
    centre1 = np.column_stack((xc, yc))
    centre = np.squeeze(centre1)
                             
    return centre, r , s

In [8]:
#function to define the mid-point of the femoral neck 
def fem_neck_mid(points):
#defining fem neck
    lat_fem_neck = list(range(32,37)) # lateral femoral neck points 32-36
    med_fem_neck = list(range(12,7,-1))  #medial femoral neck points 8-12               
    dmin = 10000

    for i in med_fem_neck:
        for j in lat_fem_neck:
            d, l, z = shortest_distance_from_point_to_segment(points [i], points [j], points [j+1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =i

    for m in lat_fem_neck:
        for n in med_fem_neck:
            d, l, z = shortest_distance_from_point_to_segment(points [m], points [n], points [n-1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =m
#print ("the intersect point z: ", zmin)

#making coordinates for the mid point of the femoral neck
    xfn = ((zmin[0] + points[index_min][0])/2)
    yfn = ((zmin[1] + points[index_min][1])/2)
#middle of femoral neck point
    fnp1 = np.column_stack((xfn, yfn))
    fnp = np.squeeze(fnp1)

    return fnp


In [9]:
#calculate distance from points to centre of best fit circle
def residual_to_centre(centre, r, point):
    #distance to centre from that point = dtc
    dtc = np.linalg.norm(point-centre)
    #residual is the dtc - the radius - for alpha angle we are only interested in the positive residuals where dtc is > r
    residual = dtc - r
    
    return dtc, residual 

In [10]:
#calculate index point for cam calculation, index point is the first point of three to deviate form the circle by greater than res_max
def cam_index_point(centre, r, points):
    points_circle = list(range(15,29))
    points_cam = list(range(29,40))

    res_max = 0

    for i in points_circle:
    #print(i)
        dtc, residual = residual_to_centre(centre, r, points[i])
    #print(dtc, residual)
        if residual > res_max:
            res_max = residual
        if (residual*-1) > res_max:
            res_max = (residual*-1)
        if res_max < 1:
            res_max = 1
#print(res_max)
        
    for j in points_cam:
        #print(j)
        # this looks for the point where three consequative ssm points leave the circle of best fit by more than the maxium residual from the noise obtained from the circle of best fit.
        dtc1, residual1 = residual_to_centre(centre, r, points[j])
        dtc2, residual2 = residual_to_centre(centre, r, points[j+1])
        dtc3, residual3 = residual_to_centre(centre, r, points[j+2])
        
        if residual1 > res_max and residual2 > res_max and residual3 > res_max :
            index_point_no = j
            index_point = points[j]
            break

    return index_point, index_point_no



In [11]:
#function to define the cam intercept point
def cam_intercept (points, index_point_no, centre, r):
    #define points a and b in coordinates - a & b suspected SSM points to overlap circle. c is the centre of cicrl
    ax=points[(index_point_no - 1),0]
    ay=points[(index_point_no - 1),1]

    bx=points[(index_point_no),0]
    by=points[(index_point_no),1]

    cx=centre[0]
    cy=centre[1]

    #compute the euclidean distance between A and B
    lab = sqrt( ((bx-ax)**2)+ ((by-ay)**2) )

    # compute the direction vector d from a to b
    dx = (bx-ax)/lab
    dy = (by-ay)/lab

    # the equation of the line ab is x = dx*t + ax, y = dy*t + ay with 0 <= t <= LAB.
    # compute the distance between the points a and e, where e is the point of ab closest the circle center (cx, cy)
    t = (dx*(cx-ax)) + (dy*(cy-ay))    

    # compute the coordinates of the point E
    ex = t*dx+ax
    ey = t*dy+ay
    # compute the euclidean distance between e and c
    lec = sqrt(((ex-cx)**2)+((ey-cy)**2))

    # test if the line intersects the circle
    if lec < r :
        # compute distance from t to circle intersection point
        dt = sqrt( (r**2) - (lec**2))
        # compute first intersection point
        fx = (t-dt)*dx + ax
        fy = (t-dt)*dy + ay

        # compute second intersection point
        gx = (t+dt)*dx + ax
        gy = (t+dt)*dy + ay
    
        #make sure you get the intersection point on the lateral side of circle
        if fx < points[(index_point_no - 1),0]:
            fx = gx
            fy = gy

    #else test if the line is tangent to circle
    elif lec == r:
        fx = ax
        fy = ay

    elif lec > r:
        dtc1, residual1 = residual_to_centre(centre, r, points[(index_point_no)])
        dtc2, residual2 = residual_to_centre(centre, r, points[(index_point_no - 1)])
        dtc3, residual3 = residual_to_centre(centre, r, points[(index_point_no + 1)])
        if residual1 < residual2 and residual1 <residual3:
            fx = points[(index_point_no),0]
            fy = points[(index_point_no),1]
        elif residual2 < residual1 and residual2 < residual3:
            fx = points[(index_point_no-1),0]
            fy = points[(index_point_no-1),1]
        elif residual3 < residual1 and residual3 < residual2:
            fx = points[(index_point_no+1),0]
            fy = points[(index_point_no+1),1]

    cam_intercept_point1 = np.column_stack((fx, fy))
    cam_intercept_point = np.squeeze(cam_intercept_point1)

    return cam_intercept_point

In [12]:
#deriving alpha angle from femoral neck point, best fit circle and cam index point (alpha - a=fnp, b centre, c=cam index point)
#(deriving lateral centre edge angle, a = point 85 b = centre, c = perpendicular point to centre,)
def angle(a, centre, c):
    b = centre

    ba = a - b
    bc = c - b
#    bc1 = np.squeeze(np.asarray(bc))

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    angle_degrees = np.degrees(angle)
    
    
    return angle_degrees

In [1]:
#calculate impingement area and export to csv
pts_files = open('IMAGE_LIST_FILE', 'r+') #imagelist file name here
f1 = open('OUTPUT_FILE_NAME','w') #file you want to write to
for pts_file in pts_files:
    print ("working on file: "+pts_file[:-1])
    points = read_points_file("PATH_TO_POINTS_FILES/"+pts_file[:-1]) #directory for where you points files are stored
    #cam measurements
    centre, r, s = fit_circle(points)
    index_point, index_point_no = cam_index_point(centre, r, points)
    cam_intercept_point = cam_intercept(points, index_point_no, centre, r)
    fnp = fem_neck_mid(points)
    #alpha angle
    alpha = angle(fnp, centre, cam_intercept_point)

    #write output file
    f1.write(str(pts_file)+","+str(alpha)+'\n')

    
f1.close()
pts_files.close()

FileNotFoundError: [Errno 2] No such file or directory: 'IMAGE_LIST_FILE'